Adding Wind Pressure Coefficients to Surfaces in an E+ IDF File using Python
==================================================================================================

Introduction
------------

In this tutorial, we will walk you through a Python script designed to add wind pressure coefficients to surfaces in an EnergyPlus IDF file. This can be particularly useful when you're modelling airflow and want to understand how the building's geometry interacts with the wind.

Prerequisites
-------------

Before we begin, ensure you have the following:

1.  Basic knowledge of Python.
2.  EnergyPlus and the `Energy+.idd` file.
3.  Python's EPPY library installed. If not, you can install it using pip: `pip install eppy`.


The `Energy+.idd` file, often simply referred to as the IDD, is the Input Data Dictionary for EnergyPlus. This file defines all the objects and fields that an IDF (Input Data File) will have. The IDD is crucial for software and scripts (like the one using EPPY in your previous query) to understand the structure of an IDF.

Here's how you can find the `Energy+.idd` file:

1.  Installation Directory: When you install EnergyPlus, the IDD is usually placed in the main installation directory of EnergyPlus.

    For example, on a Windows system, if you've installed EnergyPlus 9.3, the path might be something like:

    makefileCopy code

    `C:\EnergyPlusV9-3-0\EnergyPlusIDD.idd`

    On Linux, it might be under `/usr/local/EnergyPlus-9-3-0/` or a similar location, depending on how you've installed it.

    On macOS, if you've installed EnergyPlus using the installer package, the path might be:

    Copy code

    `/Applications/EnergyPlus-9-3-0/`

2.  EnergyPlus Toolkits: Some toolkits or GUI-based applications that work with EnergyPlus might keep their own copy of the IDD. If you're using such a toolkit, you might find the IDD in its directories.

3.  Search Your System: If you're unsure where EnergyPlus was installed, you can use system search functions:

    -   On Windows, you can use the search bar in the File Explorer.
    -   On Linux, you can use commands like `find` or `locate` (if `updatedb` has been run previously).
    -   On macOS, you can use Spotlight or Finder.
4.  Check Documentation: The EnergyPlus documentation might provide information on the installation directory, which can guide you to the location of the IDD.

When referencing the IDD in scripts or software, it's good practice to always point to the version of the IDD that matches the version of your EnergyPlus IDF files. This ensures compatibility and that all objects and fields are recognized correctly.



The Script
----------

The script you'll be working with uses the EPPY library to interact with EnergyPlus IDF files. We'll start by importing necessary modules, setting paths, and then diving into the core logic of the script.

In [49]:
# Importing necessary libraries
from eppy import modeleditor
from eppy.modeleditor import IDF

# Define paths
IDF_PATH = 'data/box_clean.idf'  # This is your EnergyPlus IDF file.
IDD_PATH = 'idd/Energy+.idd'    # This file defines all the objects and fields that an IDF will have. It is distributed with EnergyPlus.

# Load the IDF file using EPPY
IDF.setiddname(IDD_PATH)    # Setting the IDD file so EPPY knows how to read the IDF file.
idf1 = IDF(IDF_PATH)


In [50]:
REF_IDF_PATH = 'box22_reference.idf'  # This is your EnergyPlus IDF file.
idf_ref = IDF(REF_IDF_PATH)
Airflownetwork_Multizone_Component_Detailedopening = idf_ref.idfobjects['AirflowNetwork:MultiZone:Component:DetailedOpening'][0]

Airflownetwork_Multizone_Component_Detailedopening


AirflowNetwork:MultiZone:Component:DetailedOpening,
    Block1:Zone1_Wall_2_0_0_0_0_0_Win,    !- Name
    0.00014,                  !- Air Mass Flow Coefficient When Opening is Closed
    0.65,                     !- Air Mass Flow Exponent When Opening is Closed
    NonPivoted,               !- Type of Rectangular Large Vertical Opening LVO
    0,                        !- Extra Crack Length or Height of Pivoting Axis
    2,                        !- Number of Sets of Opening Factor Data
    0,                        !- Opening Factor 1
    0.65,                     !- Discharge Coefficient for Opening Factor 1
    0,                        !- Width Factor for Opening Factor 1
    0,                        !- Height Factor for Opening Factor 1
    0,                        !- Start Height Factor for Opening Factor 1
    1,                        !- Opening Factor 2
    0.65,                     !- Discharge Coefficient for Opening Factor 2
    1e-08,                    !- Width Factor

Note: Ensure both `box_clean.idf` and `Energy+.idd` are in the current working directory or provide the full path to the files.

### Wind Pressure Coefficient Array

The first step is to define the wind direction and its corresponding coefficient. This is crucial to understand how wind affects different surfaces at various angles.

In [51]:
wind_pressure_coefficient_array = idf1.newidfobject("AirflowNetwork:MultiZone:WindPressureCoefficientArray",
                               Name="Cp Data",
                               Wind_Direction_1 = 0,
                               Wind_Direction_2 = 45,
                               Wind_Direction_3 = 90,
                               Wind_Direction_4 = 135,
                               Wind_Direction_5 = 180,
                               Wind_Direction_6 = 225,
                               Wind_Direction_7 = 270,
                               Wind_Direction_8 = 315)

Here, we're setting up a new `AirflowNetwork:MultiZone:WindPressureCoefficientArray` object which describes how the wind pressure coefficient (Cp) varies with wind direction.

### Sample cp values

These are sample Cp values for each wind direction. For this tutorial, we are using a predefined set. In a real-world scenario, these values might be derived from wind tunnel testing or CFD simulations.

In [52]:
# Sample cp values
CP_values = [-0.6,-0.5,-0.4,-0.5,-0.6,-0.5,-0.4,-0.5]
building_objects = idf1.idfobjects['BUILDINGSURFACE:DETAILED']
fenestration_objects = idf1.idfobjects['FENESTRATIONSURFACE:DETAILED']

### Adding Cp values to Building Surfaces

Now, we'll loop through each building surface in the IDF. For every surface exposed to the outdoors, we'll assign the Cp values:

In [53]:
import copy
node_height = 6.5 # Placeholder
opening_array = []
for surface in fenestration_objects:
    if surface.View_Factor_to_Ground!=0:
        #print(f'Adding wind pressure coefficients for {surface.Name}')
        idf1.newidfobject("AirflowNetwork:MultiZone:WindPressureCoefficientValues",
                          Name=surface.Name,
                          AirflowNetworkMultiZoneWindPressureCoefficientArray_Name = wind_pressure_coefficient_array.Name,
                          Wind_Pressure_Coefficient_Value_1 = CP_values[0],
                          Wind_Pressure_Coefficient_Value_2 = CP_values[1],
                          Wind_Pressure_Coefficient_Value_3 = CP_values[2],
                          Wind_Pressure_Coefficient_Value_4 = CP_values[3],
                          Wind_Pressure_Coefficient_Value_5 = CP_values[4],
                          Wind_Pressure_Coefficient_Value_6 = CP_values[5],
                          Wind_Pressure_Coefficient_Value_7 = CP_values[6],
                          Wind_Pressure_Coefficient_Value_8 = CP_values[7])
        #print(f'Adding external node for {surface.Name}')
        idf1.newidfobject("AirflowNetwork:MultiZone:ExternalNode",
                            Name=surface.Name + '.EN', # Don't know why this is .EN
                            External_Node_Height = node_height,
                            Wind_Pressure_Coefficient_Curve_Name = surface.Name,
                            Symmetric_Wind_Pressure_Coefficient_Curve = 'No',
                            Wind_Angle_Type = 'Absolute')
        print(f'Adding detailed opening settings for {surface.Name}')
        new_detailed_opening = idf1.newidfobject("AirflowNetwork:MultiZone:Component:DetailedOpening")
        
        # Set properties for the DetailedOpening object:
        new_detailed_opening.Name = surface.Name
        new_detailed_opening.Air_Mass_Flow_Coefficient_When_Opening_is_Closed = 0.00014
        new_detailed_opening.Air_Mass_Flow_Exponent_When_Opening_is_Closed = 0.65
        new_detailed_opening.Type_of_Rectangular_Large_Vertical_Opening_LVO = "NonPivoted"
        new_detailed_opening.Extra_Crack_Length_or_Height_of_Pivoting_Axis = 0
        new_detailed_opening.Number_of_Sets_of_Opening_Factor_Data = 2

        # Opening Factor 1 properties
        new_detailed_opening.Opening_Factor_1 = 0
        new_detailed_opening.Discharge_Coefficient_for_Opening_Factor_1 = 0.65
        new_detailed_opening.Width_Factor_for_Opening_Factor_1 = 0
        new_detailed_opening.Height_Factor_for_Opening_Factor_1 = 0
        new_detailed_opening.Start_Height_Factor_for_Opening_Factor_1 = 0

        # Opening Factor 2 properties
        new_detailed_opening.Opening_Factor_2 = 1
        new_detailed_opening.Discharge_Coefficient_for_Opening_Factor_2 = 0.65
        new_detailed_opening.Width_Factor_for_Opening_Factor_2 = 1e-08
        new_detailed_opening.Height_Factor_for_Opening_Factor_2 = 1
        new_detailed_opening.Start_Height_Factor_for_Opening_Factor_2 = 0

        # Opening Factor 3 properties
        new_detailed_opening.Opening_Factor_3 = 0
        new_detailed_opening.Discharge_Coefficient_for_Opening_Factor_3 = 0
        new_detailed_opening.Width_Factor_for_Opening_Factor_3 = 0
        new_detailed_opening.Height_Factor_for_Opening_Factor_3 = 0
        new_detailed_opening.Start_Height_Factor_for_Opening_Factor_3 = 0

        # Opening Factor 4 properties
        new_detailed_opening.Opening_Factor_4 = 0
        new_detailed_opening.Discharge_Coefficient_for_Opening_Factor_4 = 0
        new_detailed_opening.Width_Factor_for_Opening_Factor_4 = 0
        new_detailed_opening.Height_Factor_for_Opening_Factor_4 = 0
        new_detailed_opening.Start_Height_Factor_for_Opening_Factor_4 = 0

        opening_array.append(new_detailed_opening)

idf1.idfobjects["AirflowNetwork:MultiZone:Component:DetailedOpening"] = opening_array
                            


Adding detailed opening settings for Block1:Zone1_Wall_2_0_0_0_0_0_Win
Adding detailed opening settings for Block1:Zone1_Wall_3_0_0_0_0_0_Win
Adding detailed opening settings for Block1:Zone1_Wall_4_0_0_0_0_0_Win
Adding detailed opening settings for Block1:Zone1_Wall_5_0_0_0_0_0_Win


In [54]:
idf1.newidfobject("Output:SQLite",
                  Option_Type = "Simple",
                  Unit_Conversion_for_Tabular_Data = "JtoKWH")
                  


OUTPUT:SQLITE,
    Simple,                   !- Option Type
    JtoKWH;                   !- Unit Conversion for Tabular Data

Here, we're checking each building surface to see if its boundary condition is set to 'Outdoors'. If so, we're adding the Cp values to it.

### Save Changes

Finally, let's save our modified IDF:

In [55]:
idf1.saveas('box_clean_with_cp.idf')

This command will save the modified IDF as 'box_clean_with_cp.idf'. You can now run this new IDF in EnergyPlus to get the effects of the wind pressure coefficients.

Conclusion
You've now successfully added wind pressure coefficients to the surfaces in an E+ IDF file. This script can be adapted and expanded upon based on the specific requirements of your EnergyPlus models. Happy coding and modelling!